In [1]:
from sklearn.neighbors import KDTree
import pandas as pd
import numpy as np


In [2]:
graph = 'hardrockfeatures' #Nombre del csv
features_folder = 'data/track_features/' #ruta

In [3]:
data = pd.read_csv(features_folder + graph+'.csv')

In [4]:
to_recommend = data.set_index('id').iloc[:,5:15] #OJO, ACA EL INDICE ESTA SETEADO AL ID DE LA CANCIÓN//

In [5]:
to_recommend.head() ##FILTRADO PARA CONTENER ID Y FEATURES RELEVANTES

,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
id,,,,,,,,,,
51,0.438,0,-14.522,0,0.0346,0.21600,0.000466,0.0787,0.552,108.965
64,0.340,9,-12.049,0,0.0339,0.58000,0.003200,0.1160,0.197,82.433
70,0.834,0,-8.210,1,0.0577,0.07380,0.000095,0.0924,0.338,118.223
71,0.677,4,-6.183,1,0.0343,0.00101,0.000100,0.2110,0.684,127.678
72,0.657,5,-8.868,1,0.0545,0.00983,0.007480,0.0923,0.209,118.704


In [6]:
class knn_recommender():
    
    def __init__(self):
        self.tree = None
        self.data = None
        
    def fit(self, data, metric = 'cosine'):
        """
        Genera el arbol de búsqueda para el dataset
        """
        self.data = data
        self.tree = KDTree(self.data, leaf_size=2)
        
    def recommend_for_one(self, query, k = 5):
        """
        Recomiendo los k mas cercanos de una canción
        Input: ID de la canción
        """
        
        dist, ind = self.tree.query(self.data.loc[query].to_numpy().reshape(1,-1), k)
        return self.data.index.to_numpy()[ind]
    
    def recommend(self, query_1, query_2, n): 
        """
        Query 1 es el desde, query 2 el hasta,  n la cantidad de recomendaciones
        Se divide por parte entera (mitad primera cancion, mitad segunda, redondeado hacia abajo)
        Retorna el ID de la canción // Se puede buscar el nombre con la función del otro código
        """
        k = n//2
        ind1 = self.recommend_for_one(query_1, k).reshape(k)
        ind2 = self.recommend_for_one(query_2, k).reshape(k)
        
        return np.hstack([ind1, ind2])
        
    
    

In [7]:
to_recommend.index.to_numpy()[[2,3]]

array([70, 71], dtype=int64)

In [8]:
rec =  knn_recommender()
rec.fit(to_recommend)

In [9]:
rec.recommend_for_one(51)

array([[     51, 2137094,  800169, 1819062,   94906]], dtype=int64)

In [10]:
to_recommend.index[[0, 25325]]

Int64Index([51, 2137094], dtype='int64', name='id')

In [11]:
Recomendaciones = rec.recommend(51,72, 10)

In [12]:
to_recommend.loc[Recomendaciones]

,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
id,,,,,,,,,,
51,0.438,0,-14.522,0,0.0346,0.21600,0.000466,0.0787,0.552,108.965
2137094,0.593,0,-14.152,1,0.0305,0.07130,0.000007,0.2300,0.605,108.755
800169,0.439,0,-15.164,1,0.0406,0.45400,0.000061,0.1770,0.780,109.396
1819062,0.244,0,-15.288,1,0.0409,0.75400,0.002490,0.1110,0.644,108.869
94906,0.501,1,-14.192,1,0.0318,0.16000,0.025000,0.1460,0.333,108.291
72,0.657,5,-8.868,1,0.0545,0.00983,0.007480,0.0923,0.209,118.704
1179789,0.574,5,-8.300,1,0.0383,0.39900,0.081800,0.1310,0.568,118.296
316256,0.467,5,-9.295,1,0.0314,0.46800,0.000088,0.0511,0.935,118.159
4420,0.748,4,-9.072,1,0.0363,0.12700,0.000000,0.4470,0.514,118.852
